In [20]:
# variables for the user to input

# host organization is the ArcGIS Online Org
HOST_URL = "https://envisioning.maps.arcgis.com"
HOST_USERNAME = "GBushongENV"

# guest organization is the Enterprise Portal
GUEST_URL = "https://pdoscriptdev.esri.com/portal"
GUEST_USERNAME = "admin"

# this group in the host org contains items to share or is empty
COLLAB_HOST_GROUP = "TEST-1"

# this group in the guest org contains items to share or is empty
COLLAB_GUEST_GROUP = "aec-enterprise-group"

# this will be the name of the collaboration and the name of the folder created in enterprise
COLLAB_NAME = "Enterprise + Online"

# name of the workspace that the groups will be added to
WORKSPACE_NAME = "Enterprise Workspace"

# description for the collaboration and workspace
DESCRIPTION = "Data sharing sample between Enterprise and Online"

# how many hours from sending the invitation should be valid for (integer)
EXPIRATION = 24

# host contact info
FIRST_NAME = "Grace"
LAST_NAME = "Bushong"
EMAIL = "gbushong@esri.com"

# guest contact info
GUEST_FIRST = "Grace"
GUEST_LAST = "Bushong"
GUEST_EMAIL = "gbushong@esri.com"

# local folder to temporarily save invitation and response docs into
LOCAL_TMP_DIR = "/Users/grac9792/Documents/pdo-scripts"

In [31]:
# import libraries
from arcgis.gis import GIS
from arcgis.gis.admin import Collaboration

In [3]:
# connect to guest GIS, the ArcGIS Enterprise instance
guest_gis = GIS(GUEST_URL, GUEST_USERNAME, verify_cert=False)

# connect to host GIS, the new ArcGIS Online instance
host_gis = GIS(HOST_URL, HOST_USERNAME)

Enter password: ········
Enter password: ········


# Create a Collaboration

In [21]:
# search for sample groups in host gis
host_group = host_gis.groups.search('title:{}'.format(COLLAB_HOST_GROUP))[0]
host_group

<Group title:"TEST-1" owner:GBushongENV>

In [22]:
# create a collaboration
host_collab = host_gis.admin.collaborations.create(name=COLLAB_NAME, description=DESCRIPTION,
                                                  workspace_name=WORKSPACE_NAME,
                                                  workspace_description=DESCRIPTION,
                                                  portal_group_id=host_group.id,
                                                  host_contact_first_name=FIRST_NAME,
                                                  host_contact_last_name=LAST_NAME,
                                                  host_contact_email_address=EMAIL,
                                                  access_mode='sendAndReceive')
collab_id = host_collab['id']
print(collab_id)
host_collab

2310456340dd48938656358281013872


{'id': '2310456340dd48938656358281013872',
 'name': 'Enterprise + Online',
 'description': 'Data sharing sample between Enterprise and Online',
 'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
 'config': {'ownerUsername': 'GBushongENV'},
 'created': 1544826752000,
 'modified': 1544826752000}

# Invite participant GIS to the collaboration

In [23]:
# get the list of workspaces available as part of the collaboration
host_collab.workspaces

[{'id': '6f37f40119464bac8076fd62e8870d14',
  'name': 'Enterprise Workspace',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationId': '2310456340dd48938656358281013872',
  'config': {'maxItemSizeInMB': 1024,
   'maxReplicationPackageSizeInMB': 5120,
   'copyByRefIfCopyFail': True},
  'created': 1544826752000,
  'modified': 1544826752000,
  'lastSyncSuccess': -1,
  'participantGroupLinks': [{'portalId': '9F2Q47vO7UfnIjjV',
    'portalGroupId': 'ee213017ab39402fb933d01c8bbe2e87',
    'portalGroupName': 'TEST-1',
    'config': {'enableRealtimeSync': True, 'copyFeatureServiceData': False},
    'accessMode': 'sendAndReceive',
    'canSyncWorkspace': False}],
  'participantsWithoutGroupLinks': []}]

In [24]:
# compose the list of collaboration workspaces and the privileges for each
# assume there is only one workspace in the collaboration?
config = [{host_collab.workspaces[0]['id']:'sendAndReceive'}]

# invite the guest GIS and download the invitation file
invite_file = host_collab.invite_participant(config_json=config, expiration=EXPIRATION, 
                                             guest_portal_url=GUEST_URL,
                                             save_path=LOCAL_TMP_DIR)

# print the path to the invite file
invite_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.invite'

# Accept the invitation at the participant GIS

In [25]:
guest_gis.admin.collaborations.accept_invitation(first_name=GUEST_FIRST, last_name=GUEST_LAST,
                                                email=GUEST_EMAIL,
                                                invitation_file=invite_file)

{'success': True}

In [27]:
# get the list of collaborations on the guest GIS and pick the one created earlier
guest_gis.admin.collaborations.list()

[{'id': '0375e108fa4b475390444fa503330843',
  'name': 'Enterprise + Online',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
  'config': {'ownerUsername': 'admin'},
  'created': 1544740972900,
  'modified': 1544740972900},
 {'id': 'd6e8af1167bd4137b89ee712d9806cab',
  'name': 'Manual Collaboration Test',
  'description': 'test',
  'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
  'config': {'ownerUsername': 'admin'},
  'created': 1544818473149,
  'modified': 1544818473149},
 {'id': '2310456340dd48938656358281013872',
  'name': 'Enterprise + Online',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
  'config': {'ownerUsername': 'admin'},
  'created': 1544826860649,
  'modified': 1544826860649}]

In [30]:
# assume it's the only collaboration, but might need user to be paying attention here
# enterprise can only have one collaboration though, right, so it will be the only collaboration?
#guest_collab = guest_gis.admin.collaborations.list()[0]

guest_collab = Collaboration(guest_gis.admin.collaborations, collab_id)
type(guest_collab)

{'id': '2310456340dd48938656358281013872', 'name': 'Enterprise + Online', 'description': 'Data sharing sample between Enterprise and Online', 'collaborationHostPortalId': '9F2Q47vO7UfnIjjV', 'config': {'ownerUsername': 'admin'}, 'created': 1544826860649, 'modified': 1544826860649}


arcgis.gis.admin._collaboration.Collaboration

## Export the response file

In [10]:
# now export a response file that can be sent back to the host GIS
response_file = guest_collab.export_invitation(out_folder=LOCAL_TMP_DIR)
response_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.response'

# Join a Group to the Collaboration in the guest GIS

In [11]:
# first get the list of workspaces in the guest collaboration
guest_collab.workspaces

[{'id': '11ed756d6b42422ebf46c125586622a9',
  'name': 'Enterprise sharing',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationId': '0375e108fa4b475390444fa503330843',
  'config': {'maxItemSizeInMB': 1024,
   'maxReplicationPackageSizeInMB': 5120,
   'copyByRefIfCopyFail': True},
  'created': 1544740973010,
  'modified': 1544740973010,
  'lastSyncSuccess': -1,
  'participantGroupLinks': [{'portalId': '9F2Q47vO7UfnIjjV',
    'accessMode': 'sendAndReceive',
    'config': {}}],
  'participantsWithoutGroupLinks': [{'portalId': '39dde654-828b-4a97-ad02-92e952a33ff9',
    'accessMode': 'sendAndReceive',
    'config': {}}]}]

In [12]:
#guest_group = guest_gis.groups.create(title=host_group.title,
#                                     tags=host_group.tags,
#                                     description=host_group.description,
#                                     snippet=host_group.snippet,
#                                     access=host_group.access)
guest_group = host_gis.groups.search(COLLAB_GUEST_GROUP)[0]
guest_group

aec-test-group
['aec group']
None
To be used for testing aec scripts
private


<Group title:"aec-test-group" owner:admin>

In [13]:
# again assuming it's the first workspace
guest_collab.add_group_to_workspace(portal_group=guest_group, workspace=guest_collab.workspaces[0])

{'success': True}

# Accept the collaboration response at the host GIS

In [14]:
host_collab.import_invitation_response(response_file)

{'success': True}

In [15]:
# verify the collaboration
print("Collaborations on host GIS:")
for collab in host_gis.admin.collaborations.list():
    print(collab.name)
    
print("-----------------------------")
print("Collaborations on guest GIS:")
for collab in guest_gis.admin.collaborations.list():
    print(collab.name)

Collaborations on host GIS:
Enterprise + Online
-----------------------------
Collaborations on guest GIS:
Enterprise + Online
